# Trace Event Flame Graph

How to generate a Flame Graph out of Google's Trace Event format, using Duration ('B' and 'E') and Complete ('X') events. This heatmap does not include the 'CpuProfile' Instant event profile. Example file was recorded in Chrome.

In [6]:
f = open('examples/Profile-20180724T170454_abcnews', 'r')

In [7]:
import json

data = json.load(f)

In [8]:
root = {'name': 'root', 'value': 0, 'children': []}
open_partial_slices = {}

# TODO: handle CPU time differences, where "E" comes before "B"

def get_child_slice(parent_slice, name):
    for index, child in enumerate(parent_slice['children']):
        if child['name'] == name:
            return parent_slice['children'].pop(index)
    return None

def insert_slice(parent_slice, new_slice):
    child_slice = get_child_slice(parent_slice, new_slice['name'])
    if child_slice is None:
        child_slice = {'name': new_slice['name'], 'value': 0, 'children': []}
    for child in new_slice['children']:
        insert_slice(child_slice, child)
    child_slice['value'] += new_slice['value']
    parent_slice['children'].append(child_slice)

def check_thread(pid, tid):
    if pid not in open_partial_slices:
        open_partial_slices[pid] = {}
    if tid not in open_partial_slices[pid]:
        open_partial_slices[pid][tid] = []

def begin_slice(pid, tid, cat, name, ts, tts):
    check_thread(pid, tid)
    open_partial_slices[pid][tid].append({'pid': pid, 'tid': tid, 'cat': cat, 'name': name, 'ts': ts, 'tts': tts, 'children': []})

def end_slice(pid, tid, ts, tts):
    partial_slice_count = len(open_partial_slices[pid][tid])
    if partial_slice_count > 0:
        current_slice = open_partial_slices[pid][tid].pop()
        current_slice['dur'] = ts - current_slice['ts']
        current_slice['tdur'] = tts - current_slice['tts']
        if current_slice['dur'] > 0:
            current_slice['value'] = current_slice['tdur'] / current_slice['dur']
        partial_slice_count = len(open_partial_slices[pid][tid])
        if partial_slice_count > 0:
            open_partial_slices[pid][tid][partial_slice_count - 1]['children'].append(current_slice)
        else:
            insert_slice(root, current_slice)
    else:
        raise Exception("end_slice called without an open slice")

In [9]:
start = None
end = None

# TODO: handle "sf" and "stack" properties on Duration Events

for row in data:
    if row['ph'] != 'M':
        if start is None or int(row['ts']) < start:
            start = int(row['ts'])
        if end is None or int(row['ts']) > end:
            end = int(row['ts'])
    if row['ph'] == 'B' or row['ph'] == 'E':
        if row['ph'] == 'B':
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row['tts'])
        elif row['ph'] == 'E':
            end_slice(row['pid'], row['tid'], row['ts'], row['tts'])
    elif row['ph'] == 'X':
        if 'dur' in row and row['dur'] > 0 and 'tdur' in row and row['tdur'] > 0:
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row['tts'])
            end_slice(row['pid'], row['tid'], row['ts'] + row['dur'], row['tts'] + row['tdur'])

In [10]:
import json

with open('trace_event.json', 'w') as file:
     file.write(json.dumps(root))